In [1]:
df=spark.sql("select * from user_erin.review")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,None,pyspark,idle,Link,,erin,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 机器学习：文本特征提取
- N-Gram认为语言中每个单词只与其前面长度 N-1 的上下文有关。主要分为bigram和trigram，bigram假设下一个词的出现依赖它前面的一个词，trigram假设下一个词的出现依赖它前面的两个词。在SparkML中用NGram类实现，setN(2)为bigram，setN(3)为trigram。
- pyspark不能自动结合不同n-gram的特征，所以需要用VectorAssembler来结合。在build_ngrams_wocs，它需要的输入列为您准备训练的文字项列名与结果项列名。输入经过转化后会生成输出项featuresn。
- 以下的vocabsize定为5000，featuresn会从unigram，bigram，trigram中分别获得5000个特征，经过合并后，featuresn会得到15000个特征。您可以通过改变vocabsize来改变您需要得到的文本特征数量。

In [2]:
vocabsize=5000
from pyspark.ml.feature import NGram, VectorAssembler
def build_ngrams_wocs(inputCol=["only_str","score"], n=3):
    ngrams = [
        NGram(n=i, inputCol="filtered", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]

    cv = [
        CountVectorizer(vocabSize=vocabsize,inputCol="{0}_grams".format(i),
            outputCol="{0}_tf".format(i))
        for i in range(1, n + 1)
    ]
    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]

    assembler = [VectorAssembler(
        inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
        outputCol="featuresn",
    )]
    return Pipeline(stages=ngrams + cv + idf+ assembler)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.ml.feature import Tokenizer,CountVectorizer,IDF
from pyspark.ml import Pipeline
trigramwocs_pipelineFit = build_ngrams_wocs().fit(df)
df= trigramwocs_pipelineFit.transform(df)
df = df.na.fill(0) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 机器学习：将文本类特征项和数字类特征项结合
- 运用VectorAssembler将您需要结合的列名填入，例如下面inputCols的队列里填入了5个数字类列名和文本类的特征项。生成的新列名在outputCol中，为“newnfeatures”

In [4]:
from pyspark.ml.feature import StringIndexer
va = VectorAssembler(inputCols=["user","product","helpfulnessnumerator","helpfulnessdenominator","time","featuresn"], outputCol="newnfeatures")
df= va.transform(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df=df.select("id","productid","userid","newnfeatures","score")
df.write.saveAsTable("user_erin.food_features", format="orc", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…